<a href="https://colab.research.google.com/github/gnitnaw/Programs/blob/master/Chapter3/SCO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python --version

Python 3.7.11


# Example 3.5
## Import modules

In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt
print(np.__version__)

1.19.5


## Set random seed

In [3]:
np.random.seed(120)

# Example 3.17
## Input

In [4]:
c = np.array([ -6.089, -17.164, -34.054,  -5.914, -24.721,
              -14.986, -24.100, -10.708, -26.662, -22.179])

nDim = 10
#xStart = np.array([0.04067247,0.14765159,0.78323637,0.00141368,0.48526222,
#                   0.00069291,0.02736897,0.01794290,0.03729653,0.09685870])

tree = []

def addBranch(x):
  tree.append(x)

def S(x):
  y = 0
  xSum = np.sum(x)
  for i in range(nDim):
    y = y + x[i]*(c[i]+math.log(x[i]/xSum))
  return y

def constraint_1(x): #x[0]
  return 2-2*x[1]-2*x[2]-x[5]-x[9]

def constraint_2(x): #x[3]
  return 1-2*x[4]-x[5]-x[6]

def constraint_3(x): #x[7]
  return 1-x[2]-x[6]-2*x[8]-x[9]

def constraint_4(x):
  for i in range(nDim):
    if x[i] < 0.000001:
      return False
  return True

def dimension_reduce(x):
  y = x.copy()
  y[0] = constraint_1(x)
  y[3] = constraint_2(x)
  y[7] = constraint_3(x)
  return y

def penalty(x):
  y = 0
  for i in range(nDim):
    y = y + max(-(x[i]-0.000001), 0)
  y = y + S(x)
  return y

def generate_Start() :
  xStart = dimension_reduce(np.random.rand(nDim))
  while constraint_4(xStart) == False:
    xStart = dimension_reduce(np.random.rand(nDim))
  return xStart

def sampling_level1(n) : 
  x = np.zeros((n, nDim))
  for i in range(n):
    x[i] = generate_Start()
  return x

N = 20
rho = 1
omega = 0.5
debugMode = False
MaxTry = 100
tMax = 1000



## Step 1 : Generate X1...XN via uniform sampling. Set t = 0 and N_e = ceil(N rho)

In [5]:
XGet = np.array([0.04067247,0.14765159,0.78323637,0.00141368,0.48526222,
                   0.00069291,0.02736897,0.01794290,0.03729653,0.09685870])
print(S(XGet))

Xt = sampling_level1(N)
t = 0
N_elite = math.ceil(N*rho)
modd = (N%N_elite)
p = 1.0/N_elite*modd

stopLoop = False
if debugMode :
  print(Xt[0:10])

SX = np.empty(N)
for i in range(N):
  SX[i] = penalty(Xt[i])

eliteXindex = np.argsort(SX)

-47.761090811433775


## 1st while loop (Step 2)

In [6]:
while stopLoop == False :
  
  SX = SX[eliteXindex]
  Xt = Xt[eliteXindex]
  
  # Store elite X

  # Refresh S_best and X_best (Step 3)
  S_best = SX[0]
  X_best = Xt[0]

  # Step 4 : Draw Bi

  B_elite = np.random.binomial(1, p, N_elite)

  while np.sum(B_elite) != modd:
    B_elite = np.random.binomial(1, p, N_elite)

  ## 2nd loop, 1st for loop (Step. 5)
  RS = math.ceil(N*1.0/N_elite)
  kk = 0
  for i in range(N_elite):
    R = B_elite[i] + RS  ## Step 6.
    #if R > N_elite :
    #  R = N_elite
    Y = Xt[i].copy()  ## Step 7
    
    for j in range(R) : ## Step 8
      Y_p = Y.copy()
      I = np.random.randint(N_elite, size=N_elite) ## Step 9
      for k in range(N_elite):
        while I[k] == i :
          I[k] = np.random.randint(N_elite, size=1).item()
      if debugMode :
        print("i = ", i, " I[j] = ", I[j])
      SigmaX = omega*np.linalg.norm(Xt[i]-Xt[I[j]])
      Pi = np.arange(nDim)
      np.random.shuffle(Pi)  ## Step 10
      if debugMode :
        print(i, ": ", j, ": ", I, " :", SigmaX)
      for k in range(nDim): ## Step 11
        for Try in range(MaxTry):  ## Step 12
          if Pi[k] != 0 and Pi[k] != 3 and Pi[k] != 7 :
            Y_pp = Y_p.copy()
            Y_pp[Pi[k]] = Y_p[Pi[k]] + SigmaX*np.random.randn()  ## Step 13
            Y_pp = dimension_reduce(Y_pp)
            if Y_pp[Pi[k]] > 10**(-6) and Y_pp[0] > 10**(-6) and Y_pp[3] > 10**(-6) and Y_pp[7] > 10**(-6) :
              Y_p = Y_pp.copy()
              PPY = penalty(Y_p)
              if PPY < penalty(Y) :  ## Step 14
                Y = Y_p.copy()
                addBranch([t, i, j, k, Y, PPY])
                break
      Xt[kk] = Y.copy()    ## Step 15
      SX[kk] = penalty(Xt[kk])
      kk += 1
      
  eliteXindex = np.argsort(SX)
  t = t+1
  if (t%10 == 0) : 
    print("t = ", t, " Best S = ", S_best, " X = ", X_best)
  
  if t>=tMax :
    stopLoop = True

if debugMode :
  print("=========================================")
  print(tree)

if debugMode :
  print(SX)
  print(eliteXindex)
  print(B_elite)

t =  10  Best S =  -47.39253125922261  X =  [0.03336482 0.28115043 0.60838342 0.03994049 0.39617473 0.01317714
 0.15453292 0.005671   0.02851115 0.17439035]
t =  20  Best S =  -47.607077381600185  X =  [0.06809429 0.19252557 0.69591438 0.00971456 0.43269141 0.0089935
 0.11590912 0.00703332 0.01755543 0.14603232]
t =  30  Best S =  -47.697521750135046  X =  [0.06000065 0.22222702 0.70301081 0.00342074 0.47204533 0.00310072
 0.04938789 0.02038332 0.07039751 0.08642296]
t =  40  Best S =  -47.71946272946638  X =  [0.03411519 0.22989983 0.70301081 0.00280593 0.47204533 0.00242218
 0.05068124 0.0224832  0.06309171 0.09764134]
t =  50  Best S =  -47.731594293744166  X =  [0.04540869 0.18306279 0.71113755 0.00228335 0.47900511 0.00120372
 0.03850272 0.01694774 0.03421253 0.16498693]
t =  60  Best S =  -47.738146150347326  X =  [0.03387402 0.18306279 0.7233195  0.00091567 0.47914068 0.00120372
 0.03959924 0.01708123 0.03392117 0.15215768]
t =  70  Best S =  -47.74438033414549  X =  [0.041684  